<a href="https://colab.research.google.com/github/shafiqimtiaz/GenAI4SE/blob/main/notebooks/Finetuning_CodeT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import time
import random
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
import os
from transformers import RobertaTokenizer, T5ForConditionalGeneration
import nltk

# Testing

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32100, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
text = "def greet(user): print(f'hello <extra_id_0>!')"
input_ids = tokenizer(text, return_tensors="pt").input_ids

# simply generate a single sequence
generated_ids = model.generate(input_ids, max_length=10)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

user: {user.name}


In [ ]:
text = "def return_something(): result = 1 + 4 return result"
input_ids = tokenizer(text, return_tensors="pt").input_ids

# simply generate a single sequence
generated_ids = model.generate(input_ids, max_length=10)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

result


# Data Manipulation


In [ ]:
!unzip /content/T5-Pretraining-20240304T162648Z-002.zip -d /content/Data

Archive:  /content/T5-Pretraining-20240304T162648Z-002.zip
  inflating: /content/Data/T5-Pretraining/pretrain.tsv  


In [ ]:
# Data from https://drive.google.com/drive/folders/1OyERePErjkZCV-XHiEIowZ0I8nfBFAAR

dfTrain = pd.read_table('/content/Data/train-linking.tsv', names=['method','line_links'])
dfTrain

# Method shows the relevant java method with line numbers between <>
# The line links relates the <comment></comment> block to the line numbers

,method,line_links
0,public void executeImpl ( ) <1> { <2> if ( log...,<34> <35> <37> <38> <40> <41> <42> <43> <44>
1,private void updateCharge ( ) { <1> <2> widget...,<14> <15> <16> <17> <18> <19> <20> <21> <22>
2,"public static void dump ( ) { <1> <comment> "" ...",<3> <4> <5> <6> <7> <8>
3,public static IntervalSet subtract ( IntervalS...,<59> <60>
4,@ Override <1> public void clear ( ) { <2> //i...,<11> <12> <13> <14> <15> <16> <17> <18>
...,...,...
2800,@ RequestMapping ( method = RequestMethod . PU...,<39> <40> <41> <42> <43>
2801,public static boolean isValid ( String input )...,<3>
2802,private void launchDeckPickerIfNoOtherTasks ( ...,<5> <6> <7> <8> <9>
2803,"public void onSingleTapUp ( int x , int y ) { ...",<31>


In [ ]:
dfTrain.iloc[10]['method']

'public static Collection < Method > findAnnotatedMethods ( Class < ? > type , Class < ? extends Annotation > annotation ) <1> { <2> List < Method > result = new ArrayList < > ( ) ; <3> <4> // gather all publicly available methods <5> // this returns everything, even if it\'s declared in a parent <6> for ( Method method : type . getMethods ( ) ) { <7> // skip methods that are used internally by the vm for implementing covariance, etc <8> if ( method . isSynthetic ( ) || method . isBridge ( ) || isStatic ( method . getModifiers ( ) ) ) { <9> continue ; <10> } <11> <12> <comment> " look for annotations recursively in super-classes or interfaces " </comment> <13> Method managedMethod = findAnnotatedMethod ( <14> type , <15> annotation , <16> method . getName ( ) , <17> method . getParameterTypes ( ) ) ; <18> if ( managedMethod != null ) { <19> result . add ( managedMethod ) ; <20> } <21> } <22> <23> return result ; <24> } <25>'

In [ ]:
dfTrain.iloc[10]['line_links']

'<14> <15> <16> <17> <18>'

In [ ]:
dfTest = pd.read_table('/content/Data/test-linking.tsv', names=['method','line_links'])

# Dataset Class

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.data = df.set_index('index').T.to_dict('list')
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Extract the values as strings. They are Pandas
        method = self.data[idx][0]
        line_links = self.data[idx][1]

        # Tokenize
        methodTokens = tokenizer(method, return_tensors="pt", padding='max_length', truncation=True).input_ids
        linksTokens = tokenizer(line_links, return_tensors="pt", padding='max_length', truncation=True).input_ids

        # Return the method and line_links as a tuple
        return methodTokens, linksTokens

# Training

In [ ]:
def train_my_model(train_loader,val_loader,num_epochs,model,tokenizer,criterion,optimizer,target_folder, name_experiment, with_validation=False):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("Device: {}".format(device))
  model.to(device)

  total_steps = len(train_loader)
  t1 = time.time()

  training_losses, training_accuracies = [], []
  validation_losses, validation_accuracies = [], []
  model.train()
  for epoch in range(num_epochs):
      training_loss, training_acc = [], []
      validation_loss, validation_acc = [], []

      model.train()
      for i, data in enumerate(tqdm(train_loader)):
          method, links = data[0].to(device), data[1].squeeze(1).to(device)
          method = method.squeeze()
          # Forward pass
          outputs = model(input_ids=method, labels=links)
          loss = outputs[0]
          #loss = criterion(outputs.logits, links)
          # Backprop and optimization
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          training_loss.append(loss.detach().cpu())
      training_losses.append(np.mean(training_loss))

      if with_validation and epoch%5==0:
        model.eval()
        print("--- validation")
        for i, data in enumerate(tqdm(val_loader)):
          method, links = data[0].to(device), data[1].squeeze(1).to(device)
          method = method.squeeze()
          # Forward pass
          outputs=model(input_ids=method, labels=links)
          loss = outputs[0]
          #loss = criterion(outputs.logits, links)
          validation_loss.append(loss.detach().cpu())
        validation_losses.append(np.mean(validation_loss))
      print('Epoch [{}/{}], Loss: {:.4f}'
                .format(epoch + 1, num_epochs, loss.item()))
  if with_validation:
    dict = {'Training loss': training_losses}
    dict_val = {'Validation loss': validation_losses}
    df = pd.DataFrame(dict_val)
    # saving the dataframe
    df.to_csv(os.path.join(target_folder,name_experiment+'-validation.csv'))
  else:
    dict = {'Training loss': training_losses}
    df = pd.DataFrame(dict)
    # saving the dataframe
    df.to_csv(os.path.join(target_folder,name_experiment+'-train.csv'))
  print("Saving the model in ",target_folder)
  torch.save(model.state_dict(), os.path.join(target_folder,'model-'+name_experiment+'.ckpt'))
  print("######## Training Finished in {} seconds ###########".format(time.time()-t1))

In [ ]:
def run_exp(model,tokenizer,config,datafolder,saveFolder):
  print("running experiment ",config['name'],"\n")

  #loading the data
  dfTrain = pd.read_table(os.path.join(datafolder,'train-linking.tsv'), names=['method','line_links'])
  dfTest = pd.read_table(os.path.join(datafolder,'test-linking.tsv'), names=['method','line_links'])
  # Manipulating the dataframe
  df = pd.concat([dfTrain, dfTest], axis=1)
  df = df.dropna()
  df.reset_index(drop=True, inplace=True)
  df['index'] = df.index
  df[['index'] + [col for col in df.columns if col != 'index']]


  #Creating dataset and loaders
  dataset = CustomDataset(df, tokenizer)
  train_dataset, val_dataset = torch.utils.data.random_split(dataset, [0.9, 0.1])

  #training parameters
  learning_rate = 0.001
  batch_size = config['batch_size']

  #criterion and optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

  #dataloader
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)#define train loader
  val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)#define val loader

  #setting up the training and folders were experiments will be saved
  target_folder = saveFolder
  name_experiment = config['name']
  val = True
  #training
  epochs = config['num_epochs']
  train_my_model(train_loader,val_loader,epochs,model,tokenizer,criterion,optimizer,target_folder, name_experiment, with_validation=True)

  print("Training for experiment ",config['name']," is done!")

## Testing

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
config = {'name': 'Training', 'batch_size': 8, 'num_epochs': 1}

run_exp(model,tokenizer,config,'/content/Data','/content/ResultsFolder')

running experiment  Training 

Device: cuda:0


100%|██████████| 72/72 [00:47<00:00,  1.52it/s]


--- validation


100%|██████████| 8/8 [00:01<00:00,  4.31it/s]


Epoch [1/1], Loss: 0.0361
Saving the model in  /content/ResultsFolder
######## Training Finished in 54.47875237464905 seconds ###########
Training for experiment  Training  is done!


In [ ]:
# To resest mem
import gc
torch.cuda.empty_cache()
gc.collect()

0